# Spherical ICET

In [2]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp

physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)
    
%load_ext autoreload
%autoreload 2
%autosave 180

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [166]:
from ICET_spherical import ICET

## init KITTI dataset -----------------------------------------------------------------
basedir = 'C:/kitti/'
date = '2011_09_26'
drive = '0005'
frame_range = range(150, 151, 1)
dataset = pykitti.raw(basedir, date, drive)
velo1 = dataset.get_velo(0) # Each scan is a Nx4 array of [x,y,z,reflectance]
c1 = velo1[:,:3]
c1 = c1[c1[:,2] > -1.5] #ignore ground plane
## ------------------------------------------------------------------------------------

#single distinct cluster
c1 = np.random.randn(3000,3)*tf.constant([0.3,0.04,0.3]) + tf.constant([0.,4.,0.])
c2 = np.random.randn(3000,3)*tf.constant([0.3,0.04,0.3]) + tf.constant([0.,4.,0.]) - np.array([0.1, 0.3, 0.0])
# c2 = c1 - np.array([0.1, 0.3, 0.0])

it = ICET(cloud1 = c1, cloud2 = c2,  fid = 50, draw = True)
ViewInteractiveWidget(it.plt.window)


estimated solution vector X: 
 tf.Tensor([0.1 0.3 0.  0.  0.  0. ], shape=(6,), dtype=float32)
stds: 
 tf.Tensor([0.10662948 0.00106549 0.0255875  0.0044843  0.07631556 0.00601539], shape=(6,), dtype=float32)
estimated solution vector X: 
 tf.Tensor([ 0.09088177  0.30101523  0.03226731 -0.00501806  0.03149422 -0.00282478], shape=(6,), dtype=float32)
stds: 
 tf.Tensor([0.10630993 0.00106438 0.02635844 0.00441634 0.07889022 0.00594828], shape=(6,), dtype=float32)
estimated solution vector X: 
 tf.Tensor([ 0.08381683  0.30096936  0.0422958  -0.00399998  0.06346144 -0.00371498], shape=(6,), dtype=float32)
stds: 
 tf.Tensor([0.10630614 0.00106955 0.02779579 0.00437668 0.08102326 0.00592286], shape=(6,), dtype=float32)
estimated solution vector X: 
 tf.Tensor([ 0.07113962  0.3009315   0.04950405 -0.00343051  0.1196395  -0.00376625], shape=(6,), dtype=float32)
stds: 
 tf.Tensor([0.10515884 0.00107619 0.02715474 0.00432565 0.07737441 0.00594224], shape=(6,), dtype=float32)
estimated solution v

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

In [ ]:
#test where points are inside spherical cell...
# print(it.cloud1_tensor_spherical)
maxtheta = tf.constant([[0.2],[0.7]])
maxr = tf.constant([[0.5],[2.]])

ans1 = tf.greater(it.cloud1_tensor_spherical[:,1], maxtheta)
# print(ans1)
ans2 = tf.less(it.cloud1_tensor_spherical[:,0], maxr)
# print(ans2)
combined = tf.Variable([ans1, ans2])
# print(combined)
ans3 = tf.math.reduce_all(combined, axis = 1)

print(ans3)

In [42]:
#duplicate each element of an n*1 vector 3 times
t = tf.linspace(0,5,6)[:,None]
print(t)

test  = tf.tile(t, [3,1])
# print(test)
test2 = tf.reshape(tf.transpose(tf.reshape(test, [3,-1])), [-1,1])
print(test2)
test3 = tf.reshape(tf.transpose(tf.reshape(test, [3,-1])), [-1,3])
print(test3)

tf.Tensor(
[[0.]
 [1.]
 [2.]
 [3.]
 [4.]
 [5.]], shape=(6, 1), dtype=float64)
tf.Tensor(
[[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [2.]
 [2.]
 [2.]
 [3.]
 [3.]
 [3.]
 [4.]
 [4.]
 [4.]
 [5.]
 [5.]
 [5.]], shape=(18, 1), dtype=float64)
tf.Tensor(
[[0. 0. 0.]
 [1. 1. 1.]
 [2. 2. 2.]
 [3. 3. 3.]
 [4. 4. 4.]
 [5. 5. 5.]], shape=(6, 3), dtype=float64)
